In [1]:
import pandas as pd
import numpy as np
import os
from skimage.io import imread
import gc
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
import keras
from keras.applications import VGG19
from keras.layers import Input, Dense,Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from ast import literal_eval

Using TensorFlow backend.


In [3]:
save_model_name = "hackerthon.model"
W = 250.

In [4]:
path = './data'

In [5]:
o = {}
o['name'] = []
o['image'] = []
for p in os.listdir(path):
    for filename in os.listdir(path+"/"+p):
        o['name'].append(p)
        oriimg = plt.imread(path+"/"+p+"/"+filename)
        height, width, depth = oriimg.shape
        imgScale = W/width if width>height else W/height
        newX,newY = width*imgScale, height*imgScale
        newSize = newX if newX>newY else newY
        newimg = cv2.resize(oriimg,(int(newSize),int(newSize)))
        o['image'].append(newimg.tolist())

In [6]:
data = pd.DataFrame.from_dict(o)

In [ ]:
data.head()

In [7]:
del path,o,p,filename,oriimg,height, width, depth,imgScale,newX,newY,newSize,newimg

In [8]:
gc.collect()

0

In [ ]:
data['target'] = pd.Categorical(data['name']).codes

In [9]:
m = VGG19(input_shape=(250,250,3),weights='imagenet',include_top=False)

In [10]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
__________

In [11]:
output = Flatten()(m.output)

In [12]:
ouput = Dense(25088, activation='selu')(output)

In [13]:
ouput = Dense(6272, activation='selu')(ouput)

In [14]:
ouput = Dense(1568, activation='selu')(ouput)

In [15]:
ouput = Dense(392, activation='selu')(ouput)

In [16]:
ouput = Dense(196, activation='softmax')(ouput)

In [17]:
ouput = Dense(98, activation='softmax')(ouput)

In [18]:
ouput = Dense(49, activation='softmax')(ouput)

In [19]:
ouput = Dense(18, activation='softmax')(ouput)

In [20]:
m = Model(inputs=m.inputs,outputs=ouput)

In [21]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 250, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 250, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 125, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 125, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 125, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 62, 62, 128)       0         
__________

In [22]:
m.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model_checkpoint = ModelCheckpoint(save_model_name,monitor='val_acc')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001)

In [24]:
a = data['image'].tolist()

In [25]:
from functools import reduce

In [26]:
x_train = reduce(lambda x,y: np.concatenate((np.array(x).reshape(-1,250,250,3),np.array(y).reshape(-1,250,250,3)),axis=0),a)

In [27]:
y_train = pd.get_dummies(data['name'])

In [28]:
del a,data

In [29]:
gc.collect()

736

In [ ]:
hist = m.fit(x= x_train, y= y_train, validation_split=0.33,epochs=5,batch_size=32, callbacks=[model_checkpoint,reduce_lr])

Train on 220 samples, validate on 109 samples
Epoch 1/5
128/220 [================>.............] - ETA: 11:04 - loss: 2.8859 - acc: 0.0156